In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

def detect_dirty_data(filename='bank_cleaned.csv'):
    """
    Function to detect various types of dirty data in dataset
    """
  
    try:
        df = pd.read_csv(filename)
        print(f"Dataset loaded successfully. Shape: {df.shape}")
        print(f"Columns: {list(df.columns)}\n")
    except FileNotFoundError:
        print(f"File {filename} not found!")
        return
    
    dirty_data_results = {}
    
    print("=" * 60)
    print("1. NULL / MISSING VALUES DETECTION")
    print("=" * 60)
    null_indices = []
    for col in df.columns:
        null_mask = df[col].isnull() | df[col].isna() | (df[col] == '') | (df[col] == ' ')
        if null_mask.any():
            indices = df[null_mask].index.tolist()
            null_indices.extend(indices)
            print(f"Column '{col}': Index {indices}")
    
    if not null_indices:
        print("No missing values found")
    dirty_data_results['null_missing'] = list(set(null_indices))
    
    print("\n" + "=" * 60)
    print("2. TYPO / INCONSISTENT FORMAT DETECTION")
    print("=" * 60)
    typo_indices = []
    for col in df.columns:
        if df[col].dtype == 'object':
          
            values = df[col].dropna().astype(str)
            for idx in df.index:
                if pd.notna(df.loc[idx, col]):
                    val = str(df.loc[idx, col])
                  
                    if len(val) > 1 and bool(re.search(r'[a-z]', val)) and bool(re.search(r'[A-Z]', val)):
                        if val != val.title() and not val.isupper():
                            typo_indices.append(idx)
                    
                    if re.search(r'[^a-zA-Z0-9\s\-\.\,]', val):
                        typo_indices.append(idx)
            
            if typo_indices:
                print(f"Column '{col}': Index {list(set(typo_indices))}")
    
    if not typo_indices:
        print("No typos/inconsistent formats found")
    dirty_data_results['typo_inconsistent'] = list(set(typo_indices))
    
    print("\n" + "=" * 60)
    print("3. INCONSISTENT FORMAT DETECTION")
    print("=" * 60)
    format_indices = []
    for col in df.columns:
        if df[col].dtype == 'object':
            values = df[col].dropna().astype(str)
            formats = []
            for val in values:
               
                if re.match(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', val):
                    formats.append('date')
                
                elif re.match(r'[\d\-\(\)\+\s]+', val) and len(val) > 7:
                    formats.append('phone')
                
                elif '@' in val:
                    formats.append('email')
                else:
                    formats.append('text')
            
          
            format_counts = Counter(formats)
            if len(format_counts) > 1:
                for idx in df.index:
                    if pd.notna(df.loc[idx, col]):
                        val = str(df.loc[idx, col])
                       
                        most_common_format = format_counts.most_common(1)[0][0]
                        current_format = 'text'
                        if re.match(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', val):
                            current_format = 'date'
                        elif re.match(r'[\d\-\(\)\+\s]+', val) and len(val) > 7:
                            current_format = 'phone'
                        elif '@' in val:
                            current_format = 'email'
                        
                        if current_format != most_common_format:
                            format_indices.append(idx)
                
                if format_indices:
                    print(f"Column '{col}': Index {list(set(format_indices))}")
    
    if not format_indices:
        print("No inconsistent formats found")
    dirty_data_results['inconsistent_format'] = list(set(format_indices))
    
    
    print("\n" + "=" * 60)
    print("4. DUPLICATE DATA DETECTION")
    print("=" * 60)
    duplicate_indices = df.duplicated(keep=False)
    if duplicate_indices.any():
        indices = df[duplicate_indices].index.tolist()
        print(f"Duplicate rows found at Index: {indices}")
        dirty_data_results['duplicates'] = indices
    else:
        print("No duplicate rows found")
        dirty_data_results['duplicates'] = []
    
   
    print("\n" + "=" * 60)
    print("5. MISPLACED DATA DETECTION")
    print("=" * 60)
    misplaced_indices = []
    for col in df.columns:
        if df[col].dtype == 'object':
            for idx in df.index:
                if pd.notna(df.loc[idx, col]):
                    val = str(df.loc[idx, col])
                    
                    if col.lower() in ['name', 'address', 'city', 'country'] and val.isdigit():
                        misplaced_indices.append(idx)
                   
                    elif col.lower() in ['age', 'salary', 'balance', 'amount'] and not val.replace('.', '').replace('-', '').isdigit():
                        if not val.isdigit():
                            misplaced_indices.append(idx)
    
    if misplaced_indices:
        print(f"Misplaced data found at Index: {list(set(misplaced_indices))}")
    else:
        print("No misplaced data found")
    dirty_data_results['misplaced'] = list(set(misplaced_indices))
    
   
    print("\n" + "=" * 60)
    print("6. INCONSISTENT CATEGORICAL VALUES DETECTION")
    print("=" * 60)
    categorical_indices = []
    for col in df.columns:
        if df[col].dtype == 'object':
            unique_values = df[col].dropna().unique()
            if len(unique_values) < 20:  
               
                for i, val1 in enumerate(unique_values):
                    for val2 in unique_values[i+1:]:
                      
                        if isinstance(val1, str) and isinstance(val2, str):
                            if val1.lower().replace(' ', '') == val2.lower().replace(' ', ''):
                               
                                indices1 = df[df[col] == val1].index.tolist()
                                indices2 = df[df[col] == val2].index.tolist()
                                categorical_indices.extend(indices1 + indices2)
    
    if categorical_indices:
        print(f"Inconsistent categorical values found at Index: {list(set(categorical_indices))}")
    else:
        print("No inconsistent categorical values found")
    dirty_data_results['inconsistent_categorical'] = list(set(categorical_indices))
    
    
    print("\n" + "=" * 60)
    print("7. WRONG DATA TYPE DETECTION")
    print("=" * 60)
    wrong_type_indices = []
    for col in df.columns:
        if df[col].dtype == 'object':
            for idx in df.index:
                if pd.notna(df.loc[idx, col]):
                    val = str(df.loc[idx, col])
             
                    if re.match(r'^-?\d+\.?\d*$', val.strip()):
                       
                        if any(keyword in col.lower() for keyword in ['age', 'amount', 'balance', 'salary', 'id', 'number']):
                            wrong_type_indices.append(idx)
        
        elif df[col].dtype == 'object':
            for idx in df.index:
                if pd.notna(df.loc[idx, col]):
                    val = str(df.loc[idx, col])
                    if re.match(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', val):
                        if 'date' in col.lower() or 'time' in col.lower():
                            wrong_type_indices.append(idx)
    
    if wrong_type_indices:
        print(f"Wrong data types found at Index: {list(set(wrong_type_indices))}")
    else:
        print("No wrong data types found")
    dirty_data_results['wrong_type'] = list(set(wrong_type_indices))
    
    print("\n" + "=" * 60)
    print("SUMMARY OF DIRTY DATA DETECTION")
    print("=" * 60)
    total_dirty_indices = set()
    for category, indices in dirty_data_results.items():
        if indices:
            print(f"{category.upper()}: {len(indices)} rows affected")
            total_dirty_indices.update(indices)
        else:
            print(f"{category.upper()}: No issues found")
    
    print(f"\nTotal unique rows with data quality issues: {len(total_dirty_indices)}")
    print(f"Percentage of dirty data: {len(total_dirty_indices)/len(df)*100:.2f}%")
    
    return dirty_data_results

if __name__ == "__main__":
    results = detect_dirty_data('bank_cleaned.csv')

Dataset loaded successfully. Shape: (4311, 17)
Columns: ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

1. NULL / MISSING VALUES DETECTION
No missing values found

2. TYPO / INCONSISTENT FORMAT DETECTION
No typos/inconsistent formats found

3. INCONSISTENT FORMAT DETECTION
No inconsistent formats found

4. DUPLICATE DATA DETECTION
No duplicate rows found

5. MISPLACED DATA DETECTION
No misplaced data found

6. INCONSISTENT CATEGORICAL VALUES DETECTION
No inconsistent categorical values found

7. WRONG DATA TYPE DETECTION
No wrong data types found

SUMMARY OF DIRTY DATA DETECTION
NULL_MISSING: No issues found
TYPO_INCONSISTENT: No issues found
INCONSISTENT_FORMAT: No issues found
DUPLICATES: No issues found
MISPLACED: No issues found
INCONSISTENT_CATEGORICAL: No issues found
WRONG_TYPE: No issues found

Total unique rows with data quality issues: 0
Percentage of dirt